### Spark Aggregate functions

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, sum, avg, count, first, last, min, max
spark = SparkSession.builder.appName("sparkApp").getOrCreate()

In [22]:
# Read Employees data
employee_df = spark.read.csv('Data/employees.csv', header=True, inferSchema=True)
employee_df.show()

+---------+------------+------+
|     Name| Departments|salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



In [23]:
# Count Values in a column
employee_df.select(count('Departments').alias('ValueCount')).show()

+----------+
|ValueCount|
+----------+
|        10|
+----------+



In [24]:
# Get first and last value in a column
employee_df.select(first('Departments').alias('FirstVal'), last('Departments')).show()

+------------+-----------------+
|    FirstVal|last(Departments)|
+------------+-----------------+
|Data Science|         Big Data|
+------------+-----------------+



In [25]:
# Get first and last value in a column
employee_df.select(first('Salary').alias('FirstSal'), last('Salary')).show()

+--------+------------+
|FirstSal|last(Salary)|
+--------+------------+
|   10000|        2000|
+--------+------------+



In [26]:
# Get minumum and maximun value in a column
employee_df.select(min('salary'), max('salary')).show()

+-----------+-----------+
|min(salary)|max(salary)|
+-----------+-----------+
|       2000|      20000|
+-----------+-----------+



In [27]:
# Performing Aggregation using GroupBy
employee_df.groupBy("Name", "Departments").sum().show()

+---------+------------+-----------+
|     Name| Departments|sum(salary)|
+---------+------------+-----------+
|    Sunny|    Big Data|       2000|
|   Mahesh|Data Science|       3000|
|Sudhanshu|Data Science|      20000|
|    Krish|Data Science|      10000|
|   Mahesh|    Big Data|       4000|
|    Sunny|Data Science|      10000|
|Sudhanshu|         IOT|      10000|
|    Krish|         IOT|       5000|
|Sudhanshu|    Big Data|       5000|
|    Krish|    Big Data|       4000|
+---------+------------+-----------+



In [28]:
# Performing Aggregation using GroupBy and aggregate function
# The aggregate function allows us apply multiple aggregations as well as aliasing

emp_total_salary = employee_df.groupBy('Name')\
.agg(sum('salary').alias('TotalSalary'),\
avg('salary').alias('AverageSalary'),\
count('Name').alias('CountOfNames'))

emp_total_salary.show()

+---------+-----------+------------------+------------+
|     Name|TotalSalary|     AverageSalary|CountOfNames|
+---------+-----------+------------------+------------+
|Sudhanshu|      35000|11666.666666666666|           3|
|    Sunny|      12000|            6000.0|           2|
|    Krish|      19000| 6333.333333333333|           3|
|   Mahesh|       7000|            3500.0|           2|
+---------+-----------+------------------+------------+



### Using the expression (Expr) function

In [29]:
# Expressins expr() allow us to use SQL-like expressions in our transformation logic
# Let's perform a ranking of employees
employee_df.withColumn('rank', expr("case when salary >= 5000 then 'senior' else 'junior' end")).show()

+---------+------------+------+------+
|     Name| Departments|salary|  rank|
+---------+------------+------+------+
|    Krish|Data Science| 10000|senior|
|    Krish|         IOT|  5000|senior|
|   Mahesh|    Big Data|  4000|junior|
|    Krish|    Big Data|  4000|junior|
|   Mahesh|Data Science|  3000|junior|
|Sudhanshu|Data Science| 20000|senior|
|Sudhanshu|         IOT| 10000|senior|
|Sudhanshu|    Big Data|  5000|senior|
|    Sunny|Data Science| 10000|senior|
|    Sunny|    Big Data|  2000|junior|
+---------+------------+------+------+



In [30]:
# Using expression with the select function
employee_df.select('Name', 'Departments', expr('Salary * 0.05 as TaxValue')).show()

+---------+------------+--------+
|     Name| Departments|TaxValue|
+---------+------------+--------+
|    Krish|Data Science|  500.00|
|    Krish|         IOT|  250.00|
|   Mahesh|    Big Data|  200.00|
|    Krish|    Big Data|  200.00|
|   Mahesh|Data Science|  150.00|
|Sudhanshu|Data Science| 1000.00|
|Sudhanshu|         IOT|  500.00|
|Sudhanshu|    Big Data|  250.00|
|    Sunny|Data Science|  500.00|
|    Sunny|    Big Data|  100.00|
+---------+------------+--------+



In [31]:
# We can also use the native Spark aliasing
employee_df.select('name', 'Departments', expr('Salary * 0.05').alias('TaxValue')).show()

+---------+------------+--------+
|     name| Departments|TaxValue|
+---------+------------+--------+
|    Krish|Data Science|  500.00|
|    Krish|         IOT|  250.00|
|   Mahesh|    Big Data|  200.00|
|    Krish|    Big Data|  200.00|
|   Mahesh|Data Science|  150.00|
|Sudhanshu|Data Science| 1000.00|
|Sudhanshu|         IOT|  500.00|
|Sudhanshu|    Big Data|  250.00|
|    Sunny|Data Science|  500.00|
|    Sunny|    Big Data|  100.00|
+---------+------------+--------+



In [32]:
# Using expression in filter functions
employee_df.filter(expr("Salary > 4000 and Departments = 'IOT'")).show()

+---------+-----------+------+
|     Name|Departments|salary|
+---------+-----------+------+
|    Krish|        IOT|  5000|
|Sudhanshu|        IOT| 10000|
+---------+-----------+------+



In [20]:
# Using Groupby with Expression
employee_df.groupBy('Departments')\
.agg(expr("sum(Salary) as TotalSalary"),\
expr("max(salary) as highestSalary"),\
expr("min(salary) as smallestSalary")).show()

+------------+-----------+-------------+--------------+
| Departments|TotalSalary|highestSalary|smallestSalary|
+------------+-----------+-------------+--------------+
|         IOT|      15000|        10000|          5000|
|    Big Data|      15000|         5000|          2000|
|Data Science|      43000|        20000|          3000|
+------------+-----------+-------------+--------------+



### Using the select expression (selectExpr) function

In [33]:
# The select expression allows us perform select and use expression at the same time.
# It is a short-cut to column selection and using expression
employee_df.selectExpr('Name', 'Departments', 'Salary * 0.05 as TaxValue').show()

+---------+------------+--------+
|     Name| Departments|TaxValue|
+---------+------------+--------+
|    Krish|Data Science|  500.00|
|    Krish|         IOT|  250.00|
|   Mahesh|    Big Data|  200.00|
|    Krish|    Big Data|  200.00|
|   Mahesh|Data Science|  150.00|
|Sudhanshu|Data Science| 1000.00|
|Sudhanshu|         IOT|  500.00|
|Sudhanshu|    Big Data|  250.00|
|    Sunny|Data Science|  500.00|
|    Sunny|    Big Data|  100.00|
+---------+------------+--------+

